In [1]:
import pandas as pd
from datetime import datetime
import os
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format = '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s - %(lineno)d - %(message)s',
    handlers=[logging.StreamHandler()]
)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# List of restaurants
rests = ['Shaikpet', 'Kondapur', 'Yousufguda', 'Gachibowli', 'Madhapur', 'Istahgrill Himayat Nagar']
logging.info('Analysing data for the following outlets: {}'.format(rests))

# Define output file locations
sales_filename = 'output/sales_data.csv'
orders_filename = 'output/orders_data.csv'
consumption_filename = 'output/consumption_data.csv'

# Load item mapping and recipe data
logging.info("Loading item mapping and recipe data")
recipe_df = pd.read_csv('recipe.csv', index_col=0).fillna(0)
item_mapping_df = pd.read_csv('item_mapping.csv')



2020-05-20 16:04:21,710 - root - INFO - <module> - 20 - Analysing data for the following outlets: ['Shaikpet', 'Kondapur', 'Yousufguda', 'Gachibowli', 'Madhapur', 'Istahgrill Himayat Nagar']
2020-05-20 16:04:21,728 - root - INFO - <module> - 28 - Loading item mapping and recipe data


In [ ]:
# Load orders file
file = input('Enter orders file name: ')
logging.info('Filename: {}'.format(file))

data_df = pd.read_csv(file)
data_df['Time'] = pd.to_datetime(data_df['Ordered time'].apply(lambda x: x.split(' +')[0]))
start_date = min(data_df['Time'])
end_date = max(data_df['Time'])
logging.info('Given file - Minimum ordered time: {} | Maximum ordered time: {}'.format(start_date, end_date))
logging.info('=== Choose your start and end times for analysis accordingly ===')

In [ ]:
start_time = input('Enter start_time (2019-01-31 08:00:00): ')
end_time = input('Enter start_time (2019-01-31 11:59:59): ')
logging.info('SELECTION: Start Time: {} | End Time: {}'.format(start_time, end_time))

In [ ]:
if start_time > end_time:
    logging.error('Start time ahead of end time. Kindly cross check start and end times.')


In [ ]:
import psycopg2
import pandas as pd
import pytz
import datetime
import logging

logging.basicConfig(
    level=logging.INFO,
    format = '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s - %(lineno)d - %(message)s',
    handlers=[logging.StreamHandler()]
)

start_time = input('Enter start time (2019-01-31 08:00:00): ')
end_time = input('Enter end time (2019-01-31 11:59:59): ')
logging.info('SELECTION: Start Time: {} | End Time: {}'.format(start_time, end_time))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

if start_time > end_time:
    logging.error('Start time ahead of end time. Kindly cross check start and end times.')
    

database = 'istah_production'
username = 'koc_reader'
password = 'koc_center'
host = 'istahproduction.cd5d9wagzlzq.us-east-2.rds.amazonaws.com'
port = '5432'


conn = psycopg2.connect(
    database=database,
    user=username,
    password=password,
    host=host,
    port=port
)

brand = 'Istahgrill'

def get_utc(dt):
    naive = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    tz = pytz.timezone('Asia/Calcutta')
    local_dt = tz.localize(naive, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt

def getMeal(time):
    if time.hour>=7 and time.hour<11:
        return "BF"
    elif time.hour>=11 and time.hour<15:
        return "L"
    elif time.hour>=15 and time.hour<19:
        return "S"
    elif time.hour>=19 and time.hour<22:
        return "D"
    else:
        return "LN"

query = '''
    SELECT online_orders.type, online_orders.status, online_orders.bill, online_orders.gst, online_orders.number, online_orders.ordered_time, online_orders.discount, online_orders.order_type, online_orders.note, online_orders.online_customer_id, online_orders.area, online_orders.distance, 
    online_line_items.quantity, online_line_items.price, 
    variants.name as variant, 
    outlets.name as outlet_area, outlets.brand, 
    online_ratings.rating, online_ratings.issues, online_ratings.customer_comment, 
    online_customers.name as online_customer_name, online_customers.lat, online_customers.lng, online_customers.area as online_customer_area, 
    online_statuses.mfr_message, online_statuses.cancel_reason, online_statuses.payout_status
    FROM online_orders 
    LEFT JOIN online_line_items ON online_orders.id = online_line_items.online_order_id 
    LEFT JOIN variants ON online_line_items.variant_id = variants.id 
    LEFT JOIN restaurants ON online_orders.restaurant_id = restaurants.id
    LEFT JOIN outlets ON restaurants.outlet_id = outlets.id
    LEFT JOIN online_ratings ON online_orders.id = online_ratings.online_order_id
    LEFT JOIN online_customers ON online_orders.online_customer_id = online_customers.id
    LEFT JOIN online_statuses ON online_orders.id = online_statuses.online_order_id 
    WHERE ordered_time BETWEEN '{}' AND '{}' AND outlets.brand = '{}'
    '''.format(get_utc(start_time), get_utc(end_time), brand)

data = pd.read_sql(query, con=conn)

if len(data.index) != 0:
    display('Finished fetching data for {} from {} to {}'.format(brand, start_time, end_time))
    data['ordered_time'] = data['ordered_time'].dt.tz_localize('utc').dt.tz_convert('Asia/Calcutta').dt.tz_localize(None)
    #data['placed_time'] = data['placed_time'].dt.tz_localize('utc').dt.tz_convert('Asia/Calcutta').dt.tz_localize(None)
    #data['created_at'] = data['created_at'].dt.tz_localize('utc').dt.tz_convert('Asia/Calcutta').dt.tz_localize(None)
    #data['updated_at'] = data['updated_at'].dt.tz_localize('utc').dt.tz_convert('Asia/Calcutta').dt.tz_localize(None)
    data['meal_time'] =  data['ordered_time'].apply(getMeal)
    data['issues_list'] = data['issues'].fillna('').apply(lambda x: x.split(', '))
    issue_cols = data.columns
    data = pd.DataFrame(data['issues_list'].values.tolist(), index=data.index).merge(data, left_index=True, right_index=True).melt(id_vars=issue_cols, value_name='issue').drop('variable', axis=1).dropna(subset=['issue'])
    data = data.drop(['issues_list'], axis='columns')
    data['shift'] = data['ordered_time'].apply(lambda x: 'M' if 8 <= x.hour <= 18 else 'E')
else:
    display('No matching entries found for {} from {} to {}'.format(brand, start_time, end_time))

conn.close()




In [ ]:
data[data['status'] == 'cancelled'].head()

In [ ]:
data.status.unique()

In [ ]:
orders = data[['brand', 'outlet_area', 'number', 'type', 'ordered_time', 'meal_time', 'shift', 'bill', 'discount', 'order_type', 'area', 'distance', 'lat', 'lng', 'rating', 'issues', 'customer_comment']].drop_duplicates()
orders.head()



In [ ]:
# Pivot for sales data
print('Outlet wise sales breakdown'.upper())
overall_sale = orders.groupby(['outlet_area', 'type', ])['bill'].sum().unstack(level=-1)
overall_sale.loc[:,'Total'] = overall_sale.sum(axis='columns')
overall_sale.loc['Total',:] = overall_sale.sum(axis='index')
overall_sale



In [ ]:
print('Outlet + Meal wise sale breakdown'.upper())

sale_by_meal = orders.groupby(['outlet_area', 'meal_time'])['bill'].sum().unstack(level=-1).fillna(0).loc[:,['LN','BF','L','S','D']]
sale_by_meal.loc['Total', :] = sale_by_meal.sum(axis='index')
sale_by_meal


In [ ]:
print('Discount by outlet + platform')
orders.groupby(['outlet_area', 'type'])['discount'].sum().unstack(level=-1)




In [ ]:
print('AOV')
daily_sale = orders.groupby(['outlet_area', 'meal_time'])['bill'].sum().unstack(level=-1)
daily_orders = orders.groupby(['outlet_area', 'meal_time'])['number'].nunique().unstack(level=-1)

round(daily_sale.div(daily_orders, axis='index')[['LN','BF','L','S','D']])


In [ ]:

# Split items to entities and melt the entities to single rows
online_orders_df = sales_data.copy()
online_orders_df['Entity List'] = online_orders_df['Items'].fillna('').apply(lambda x:x.split('; '))
cols = online_orders_df.columns
online_orders_df = pd.DataFrame(online_orders_df['Entity List'].values.tolist(), index=online_orders_df.index).merge(online_orders_df, left_index=True, right_index=True).melt(id_vars=cols, value_name='Entity').drop('variable', axis=1).dropna(subset=['Entity'])

# Function to split an entity into Name, Quantity & Price
def split_entity(entity):
    i = entity.rfind('-')
    entity = entity[:i] + 'x' + entity[i+1:]
    split = entity.split(' x ')
    return split

# Get online orders with 
online_orders_df['Entity Split'] = online_orders_df['Entity'].fillna('').apply(split_entity)
entity_split_online_orders_df = pd.DataFrame(online_orders_df['Entity Split'].values.tolist(), index=online_orders_df.index)
entity_split_online_orders_df = entity_split_online_orders_df[[0,1,2]].rename(columns={0:'Name', 1:'Quantity', 2:'Cart Price'})
entity_split_online_orders_df = entity_split_online_orders_df['Cart Price'].fillna('').apply(lambda x: x.split(' (')[0])


# Zomato order addons do not have quantity. So initialise them to 1.0
entity_split_online_orders_df.loc[entity_split_online_orders_df['Quantity'] == '', 'Quantity'] = 1.0
entity_split_online_orders_df[['Quantity', 'Cart Price']] = entity_split_online_orders_df[['Quantity', 'Cart Price']].apply(pd.to_numeric)

# Merge entities with online orders
online_orders_df = online_orders_df.merge(entity_split_online_orders_df, left_index=True, right_index=True)
online_orders_df = online_orders_df.drop(['Entity List', 'Entity', 'Entity Split'], axis=1)
online_orders_df = online_orders_df.merge(item_mapping_df, how='left', on='Name')

# Save sales with items to csv
online_orders_df.to_csv(sales_filename, index=False)



In [ ]:
data.columns

In [ ]:
online_orders_df = data.merge(item_mapping_df, how='left', left_on='variant', right_on='Name')

In [ ]:
online_orders_df.head()

In [ ]:
# Items in mapping but not in recipe
print('Items in mapping but not in recipe'.upper())
items_not_in_recipe = item_mapping_df['Item'][(~item_mapping_df['Item'].isin(recipe_df.index)) & (item_mapping_df['Menu Item'] == True)].unique().tolist()
items_not_in_recipe



In [ ]:
# Items not getting tracked by item mapping
print('Items not getting tracked in the item mapping'.upper())
items_not_getting_tracked = online_orders_df['Name'][(online_orders_df['Item'].isna()) & (online_orders_df['Menu Item'].isna())].unique().tolist()

items_not_getting_tracked


In [ ]:
# Filter orders by menu items
online_orders_df = online_orders_df[online_orders_df['Menu Item'] == True]

# Get consumption from orders pivot and recipe
items_by_rest = online_orders_df.groupby(['outlet_area', 'Item'])['quantity'].sum().unstack(level=0).fillna(0)
items_by_rest['Total'] = items_by_rest.sum(axis='columns')
items_by_rest = items_by_rest.sort_values('Total', ascending=False)


# Save item wise orders to csv
items_by_rest.to_csv(orders_filename)
items_by_rest




In [ ]:
items_to_track = recipe_df.index.intersection(items_by_rest.index)
consumption_df =  items_by_rest.loc[items_to_track, :].transpose().dot(recipe_df.loc[items_to_track, :])

# Save consumption to csv
consumption_df.to_csv(consumption_filename)

consumption_df

In [ ]:
print('Items by meal')

items_by_meal = online_orders_df.groupby(['Item', 'Meal'])['Quantity'].sum().unstack(level=1).loc[:,['BF','L','S','D','LN1','LN2']]

items_by_meal['Total'] = items_by_meal.sum(axis='columns')

items_by_meal = items_by_meal.sort_values('Total', ascending=False)
items_by_meal['Total %'] = round(items_by_meal['Total']*100/items_by_meal['Total'].sum())
items_by_meal



In [ ]:
items_to_track = recipe_df.index.intersection(items_by_meal.index)
consumption_by_meal =  items_by_meal.loc[items_to_track, :].transpose().fillna(0).dot(recipe_df.loc[items_to_track, :])

# Save consumption to csv
# consumption_df.to_csv(consumption_filename)

consumption_by_meal = consumption_by_meal.transpose()
consumption_by_meal.transpose()